In [1]:
import numpy as np
import pandas as pd

from data_proc import proc_json

In [2]:
df = pd.read_csv(
    "./sentiment_data/LMD_data_all_returns.csv"
)

df_compnay = pd.read_csv(
    "./hackathon_data/companies.csv"
)

In [3]:
df_compnay.head()

,Ticker symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK
0,A,Agilent Technologies Inc,reports,Health Care,Health Care Equipment,"Santa Clara, California",6/5/2000,1090872
1,AAL,American Airlines Group,reports,Industrials,Airlines,"Fort Worth, Texas",3/23/2015,6201
2,AAP,Advance Auto Parts,reports,Consumer Discretionary,Automotive Retail,"Roanoke, Virginia",7/9/2015,1158449
3,AAPL,Apple Inc.,reports,Information Technology,"Technology Hardware, Storage & Peripherals","Cupertino, California",11/30/1982,320193
4,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/2012,1551152


In [4]:
df = proc_json.load_individual_transcript(company="AAPL", path="./hackathon_data/company_transcripts/")
title = df["title"]["3022"]
date = df["date"]["3022"]
body = df["body"]["3022"]

In [5]:
df["title"].values()

dict_values(['Apple (AAPL) Timothy Donald Cook on Q2 2016 Results - Earnings Call Transcript', 'Apple, Inc. (AAPL) CEO Tim Cook on Q2 2019 Results - Earnings Call Transcript', 'Apple (AAPL) Q1 2017 Results - Earnings Call Transcript', 'Apple Inc. (AAPL) CEO Tim Cook on Q4 2019 Results - Earnings Call Transcript', "Apple's CEO Discusses F1Q 2014 Results - Earnings Call Transcript", "Apple's (AAPL) CEO Tim Cook On Q3 2014 Results - Earnings Call Transcript", "Apple's CEO Discusses F1Q 2013 Results - Earnings Call Transcript", 'Apple (AAPL) Q3 2017 Results - Earnings Call Transcript', "Apple's (AAPL) CEO Tim Cook On Q4 2014 Results - Earnings Call Transcript", 'Apple (AAPL) Q2 2018 Results - Earnings Call Transcript', "Apple's CEO Discusses F2Q13 Results - Earnings Call Transcript", "Apple's CEO Discusses Q2 2012 Results - Earnings Call Transcript", "Apple's CEO Discusses F3Q12 Results - Earnings Call Transcript", "Apple's CEO Discusses Q1 2012 Results - Earnings Call Transcript", "Apple'

In [7]:
company_lst = df_compnay["Ticker symbol"].values

def qa_check(sentence: str) -> bool:
    sentence = sentence.lower()
    if len(sentence) > 100:
        return False
    if "questions" in sentence and "answers" in sentence:
        return True
    if "question" in sentence and "answer" in sentence:
        return True
    return False

In [17]:
ds_dis = dict()
ds_qa = dict()

places, length = list(), list()
failed = 0
for company in company_lst:
    all_transcript = proc_json.load_individual_transcript(company=company, path="./hackathon_data/company_transcripts/")
    for transcript_id in all_transcript["title"].keys():
        unique_id = company + "_" + transcript_id

        title = all_transcript["title"][transcript_id]
        date = all_transcript["date"][transcript_id]
        body = all_transcript["body"][transcript_id]
        # Total length of body.
        length.append(len(body))

        qa_begins = None
        for p, sentence in enumerate(body):
            if qa_check(sentence):
                qa_begins = p
                break
        if qa_begins is None:
            # If we cannot find such QA identifier.
            # Assume 0.3 Speech + 0.7 QA (aggregate prior).
            qa_begins = int(len(body) * 0.3)
            failed += 1
        places.append(qa_begins)
        # Split the dataset
        ds_dis.update({str(unique_id): " ".join(body[:qa_begins])})
        ds_qa.update({str(unique_id): " ".join(body[qa_begins:])})

In [19]:
len(ds_dis)

12379

In [139]:
print(np.mean(np.array(places) / np.array(length)))

0.3051365477581242


In [140]:
print(failed)

46
